In [1]:
import requests
import time
import datetime
from bs4 import BeautifulSoup

In [2]:
#Letterboxd
def pelicula(titulo):
  
    nombre_formateado=titulo.lower().replace(' ','-')
    url='https://letterboxd.com/film/'+nombre_formateado
    return url

def coger_nota(promedio_puntuacion):
    try:
        # Realizar split y convertir el primer elemento a float
        nota = float(promedio_puntuacion.split()[0])
        
        # Multiplicar por 2
        nota_multiplicada = nota * 2

        return nota_multiplicada
    except ValueError:
        # Manejar posibles errores de conversión
        print("Error al convertir la nota a número.")
        return None
def coger_critica(soup):
    # Supongamos que las críticas están dentro de un div con la clase "criticas"
    criticas_section = soup.find('section', class_='film-recent-reviews')

    if criticas_section:
        # Extraer todas las críticas (puedes limitar la cantidad según tus necesidades)
        criticas = criticas_section.find_all('p')

        # Devolver la primera crítica (puedes ajustar esto según tus necesidades)
        if criticas:
           
            return '\n'.join([critica.text.strip() for critica in criticas])


    return None

In [3]:
#IMDB
imdb='https://www.imdb.com'
headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',}
def buscar(titulo):
    tit_format=titulo.lower().replace(' ','%20')
    urlb=imdb+f'/find/?q={tit_format}'
    b=requests.get(urlb, headers=headers)
    sopa=BeautifulSoup(b.content, 'html.parser')
    entradas=sopa.find_all('a', class_='ipc-metadata-list-summary-item__t')
    for entrada in entradas:
        if entrada is not None:
            if titulo.lower() in entrada.text.lower():
                return imdb+entrada.get('href') #título y url
    return None

def coger_notaI(url):  #coge la nota de una pelicula, previamente ha sido seleccionada la url de la pelicula
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    nota=sopa.find('span', {'class': 'sc-bde20123-1 cMEQkK'})
    return float(nota.text)


def coger_criticaI(url):  #En ocasiones las críticas no tienen nota, hay que solucionar eso+
    r=url.split('/')
    del r[-1]
    cleaned_url='/'.join(r)
    reviews_url=cleaned_url+'/reviews'
    c=requests.get(reviews_url, headers=headers)
    sopa=BeautifulSoup(c.content, 'html.parser')
    padre=sopa.find('span', class_='rating-other-user-rating')
    if padre is not None:
        nota=padre.find('span', class_=None)
        calif=nota.text+'/10'
    else:
        calif='Unrated'
    titulo_critica=sopa.find('a', class_='title')
    critica=sopa.find('div', class_='text show-more__control')
    return calif, titulo_critica.text, critica.text

In [4]:
#FilmAffinity
def buscar_peliF(titulo):
    tit_format=titulo.lower().replace(' ','+')
    urlb=f'https://www.filmaffinity.com/us/search.php?stext={tit_format}'
    b=requests.get(urlb, headers=headers)
    sopa=BeautifulSoup(b.content, 'html.parser')
    all_div_etiq=sopa.find_all('div', class_='movie-card mc-flex movie-card-1')
    for div in all_div_etiq:
            a=div.find('a')
            if titulo.lower() in a['title'].lower():
                return a['href'],div['data-movie-id']
    return None
def coger_notaF(url):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    nota = sopa.find(id="movie-rat-avg")
    valor = nota.get_text(strip=True)
    nota=valor.replace(',','.')
    return float(nota)
def coger_criticaF(url,id):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    critica_1=sopa.find('div',itemprop='reviewBody')
    urlc=f'https://www.filmaffinity.com/es/reviews/1/{id}.html'
    src=requests.get(urlc, headers=headers)
    sopa2=BeautifulSoup(src.content, 'html.parser')
    critica_2=sopa2.find('div', class_='review-text1')
    nota=sopa2.find('div', class_='user-reviews-movie-rating')
    return  critica_1.text,critica_2.text, nota.text
def imagen(url):
    a=requests.get(url, headers=headers)
    sopa=BeautifulSoup(a.content, 'html.parser')
    img= sopa.find('img', itemprop="image")
    urli=img['src']
    response = requests.get(urli)
    display(Image(response.content))

In [5]:
def buscarPeli():

    headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',

    }
    titulo=input('Nombre de la película:')
    #Letterboxd
    url=pelicula(titulo)
    a=requests.get(url, headers=headers)
    soup=BeautifulSoup(a.content, 'html.parser')
    # Extraer información de los metadatos
    tituloPeli = soup.find('meta', property='og:title')['content']
    director_element = soup.find('span', class_='prettify')
    director_name = director_element.text.strip()
    descripcion = soup.find('meta', property='og:description')['content']
    promedio_puntuacion = soup.find('meta', {'name': 'twitter:data2'})['content']
    print(f'Título: {tituloPeli}')
    print(f"Director: {director_name}")
    print(f'Descripción: {descripcion}')
    
    #letterboxd
    print('LETTERBOXD')
    print('Nota de Letterboxd:',coger_nota(promedio_puntuacion))
    print('Críticas de usuarios de Letterboxd:',coger_critica(soup))
    #IMDb
    print('IMDB')
    urlb=buscar(titulo)
    print('Nota de IMDB',coger_notaI(urlb))
   
    
    d,e,f=coger_criticaI(urlb)
    print('Criticas de IMDB',d,e,f)
    
    print('\n')
    
    #FilmAffinity
    print('FILMAFFINITY')
    urlf,idf=buscar_peliF(titulo)
    print('Nota de FilmAffinity',coger_notaF(urlf))
    g,h,i=coger_criticaF(urlf,idf)
    print(f'Critica de FilmAffinity \n Critica de experto \n {g} \n\n Critica de usuario \n {h} ')
    
    #NOTAMEDIA
    nota_media=(coger_nota(promedio_puntuacion)+coger_notaI(urlb)+coger_notaF(urlf))/3
    print(f'La nota media de las tres páginas web es: {nota_media:.2f}')
    

In [6]:
buscarPeli()

Nombre de la película:La Dolce Vita
Título: La Dolce Vita (1960)
Director: Federico Fellini
Descripción: Journalist and man-about-town Marcello struggles to find his place in the world, torn between the allure of Rome's elite social scene and the stifling domesticity offered by his girlfriend, all the while searching for a way to become a serious writer.
LETTERBOXD
Nota de Letterboxd: 8.54
Críticas de usuarios de Letterboxd: Review by Rida   ★★★★½   20
Slight spoilers in the last paragraph.
Our modern malaise is all-encompassing self-pity. I went through it last year: what was the point of wanting things if you were never going to get them anyway? In the throes of my self-absorbed sorrow, I didn’t notice that I had no ambition to begin with. Finding something to care about got me out of that horrible dark place, but every day, I come across more people my age – kids who haven’t even hit twenty yet – diagnosed with depression, although nothing monumental has happened to them, to us. Why

Nota de FilmAffinity 7.7
Critica de FilmAffinity 
 Critica de experto 
 "'La dolce vita' (...) se levanta en medio como la piedra angular de una manera de concebir y entender el mundo. (...) la exaltación erótica de la misma decadencia, de la muerte."  

 Critica de usuario 
 Uno de los grandes clásicos de Fellini. Película sombría y cáustica, siempre me había parecido excesiva, tan recargada que el resultado final quedaba mermado a pesar de ser indiscutiblemente buena.

En el último visionado (hace unos cinco días) he cambiado de opinión. Me fascina como siempre pero esta vez no hay cansancio, sigo pensando que sobran minutos, lo que quiere decir queda claro y algún momento es redundante pero esta vez el metraje no se resintió (incluso el que me parece superfluo por reiterativo) y todo tiene la misma fuerza. De todas formas quizás la próxima vez diga lo contrario, esto con Fellini es normal.

Por lo demás esta peli la conocemos todos: cine profundamente moral, dividido en episodios si

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
from bs4 import BeautifulSoup

def buscarPeli(nombre_pelicula):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
    }

    #Letterboxd
    url_letterboxd = pelicula(nombre_pelicula)
    a = requests.get(url_letterboxd, headers=headers)
    soup = BeautifulSoup(a.content, 'html.parser')
    
    # Extraer información de los metadatos
    titulo_pelicula = soup.find('meta', property='og:title')['content']
    director_element = soup.find('span', class_='prettify')
    director_name = director_element.text.strip()
    descripcion = soup.find('meta', property='og:description')['content']
    promedio_puntuacion = soup.find('meta', {'name': 'twitter:data2'})['content']
    
    # Presentar la información
    display(HTML(f'<h2>{titulo_pelicula}</h2>'))
    display(HTML(f"<p>Director: {director_name}</p>"))
    display(HTML(f"<p>Descripción: {descripcion}</p>"))
    
    # Letterboxd
    print('LETTERBOXD')
    print('Nota de Letterboxd:', coger_nota(promedio_puntuacion))
    print('Críticas de usuarios de Letterboxd:', coger_critica(soup))
    
    # IMDb
    print('IMDB')
    url_imdb = buscar(nombre_pelicula)
    print('Nota de IMDB:', coger_notaI(url_imdb))
    d, e, f = coger_criticaI(url_imdb)
    print('Críticas de IMDB:', d, e, f)
    
    # FilmAffinity
    print('FILMAFFINITY')
    url_filmaffinity, id_filmaffinity = buscar_peliF(nombre_pelicula)
    print('Nota de FilmAffinity:', coger_notaF(url_filmaffinity))
    g, h, i = coger_criticaF(url_filmaffinity, id_filmaffinity)
    print(f'Crítica de FilmAffinity: \n Crítica de experto \n {g} \n\n Crítica de usuario \n {h} ')
    
    # NOTAMEDIA
    nota_media = (coger_nota(promedio_puntuacion) + coger_notaI(url_imdb) + coger_notaF(url_filmaffinity)) / 3
    print(f'La nota media de las tres páginas web es: {nota_media:.2f}')

# Widget de entrada
input_pelicula = widgets.Text(value='', description='Nombre de la película:')
boton_buscar = widgets.Button(description='Buscar')

# Función para manejar el evento de clic en el botón
def on_button_click(b):
    buscarPeli(input_pelicula.value)

# Asocia la función al evento de clic en el botón
boton_buscar.on_click(on_button_click)

# Muestra los widgets
display(input_pelicula)
display(boton_buscar)
